# Map of Dyanmic Tiled COG with colorbar

In [ ]:
import requests
import folium
import json
import pystac
import branca
from branca.colormap import LinearColormap

In [ ]:
tif = "https://sentinel-s1-rtc-indigo.s3.us-west-2.amazonaws.com/tiles/RTC/1/IW/12/S/YJ/2016/S1B_20161121_12SYJ_ASC/Gamma0_VV.tif"

In [ ]:
import xarray as xr
import numpy as np
da = xr.open_dataarray(tif, engine='rasterio', open_kwargs=dict(overview_level=3)).squeeze()
da = np.sqrt(da)
da.plot(cmap='gray', vmin=0, vmax=1);

In [ ]:
cm = branca.colormap.linear.Greys_06.scale(0,1)
cm.colors.reverse()
cm = branca.colormap.LinearColormap(colors=cm.colors, caption='sqrt(gamma0)')
cm

In [ ]:
m = folium.Map(
    location=(39.0666, -107.9367),
    tiles="cartodbpositron",
    zoom_start=9,
    control_scale = True,
)

# Add a public COG custom TIF layer using TiTiler (thanks devseed!)
tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
expression = "expression=sqrt(b1)&rescale=0,1"
virtual_tiles = f"{tiler}?url={tif}&{expression}"
folium.TileLayer(tiles=virtual_tiles, overlay=True, name='Indigo RTC', attr="IndigoAg").add_to(m)

folium.LayerControl().add_to(m)

# TODO: figure out positioning of this colorbar
# - Log scale?
# - Directly map matplotlib colorbar to branca
# White background! https://github.com/python-visualization/branca/issues/91
svg_style = '<style>svg {background-color: white;}</style>'
m.get_root().header.add_child(folium.Element(svg_style))
cm.add_to(m)

m

In [ ]:
m.save("raster-colorbar.html")